In [ ]:
import torch.nn as nn
from torch.distributions import Normal
import numpy as np

import sys; sys.path.append("..")
from tpp_trace import *
import utils as u
import tensor_ops as tpp

## First with no plates

In [ ]:
#- a factorised approx posterior
#- generate e.g. 3 variables
#- samples along the chain

draws = 2
nProtected = 2
data = {"data": {"__c": 10}}

# a -> b -> c observed
def chain_dist(trace):
    a = trace["a"](WrappedDist(Normal, t.ones(3), 3))
    b = trace["b"](WrappedDist(Normal, a, 3))
    c = trace["c"](WrappedDist(Normal, b, 3))
    
    return c

"""
def chain_dist(trace):
    a = trace["a"](WrappedDist(Normal, t.ones(3), 3))
    b = trace["b"](WrappedDist(Normal, a, 3))
    c = trace["c"](WrappedDist(Normal, b, 3))
    (c,) = trace.delete_names(("a", "b"), (c,))
    d = trace["d"](WrappedDist(Normal, c, 3))
    
    return d
"""
P = chain_dist


# call sampler on Q. 
# gives you the samples and a log Q tensor `log_prob`
tr1 = sampler(draws, nProtected, data=data)

val = P(tr1)
log_q = tr1.trace.out_dicts["log_prob"]

# compute the log_probs

# pass these to evaluator, which does a lookup for all the latents 
# gives you log P for each latent
tr2 = evaluator(tr1, nProtected, data=data)
val = P(tr2)

#tr2.trace.out_dicts["log_prob"]
#log_p = 

#Q = pytorch.module
#    - factorised Gaussians with learned means and covs
#    - `q.forward()` will look like chain_dist
    

#- optimise it
